In [180]:
from __future__ import print_function

import os
import base64
import pandas as pd
import json as json
import numpy as np

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from google.oauth2 import service_account

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [181]:
def calculate_calorie_percent(x,y):
    return (x-y)/x*100

def calculate_calorie_diff(x,y):
    return (x-y)

In [182]:

def make_data_frame_from_list(list):
    cols = list.pop(0)
    df = pd.DataFrame(data=list, columns=cols)
    dfCols = df.columns

    # Set the Date column to be date time dType
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

    string_cols = ['Programme', 'Phase']
    ignore_cols = ['Date'] + string_cols

    ## Go through columns we want to convert to numbers
    cols_to_num = [i for i in df.columns if i not in ignore_cols]
    for col in cols_to_num:
        df[col] = df[col].apply(pd.to_numeric, errors='coerce')

    ## Go through columns we want to conver to strings
    for col in string_cols:
        df[col] = df[col].astype(str)

    # Set the index of the data frame to be a date
    df.set_index(['Date'], inplace=True)

    df['Calories Percent'] = df.apply(lambda x: calculate_calorie_percent(x['Calories Consumed'], x['Calories Burned']), axis=1)
    df['Maintenance Calorie Diff'] = df.apply(lambda x: calculate_calorie_diff(x['Calories Consumed'], x['Calories Burned']), axis=1)

    return df

In [183]:
# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

SERVICE_ACCOUNT_FILE = 'keys-local.json'

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

SAMPLE_SPREADSHEET_ID = '1ohNM7O8Ecg1EtI3WLv3MbyuKLP7gm7WQFufcqylxeUQ'
# SAMPLE_RANGE_NAME = 'Daily Log Test!A1:N498'
SAMPLE_RANGE_NAME = 'Daily Log!A1:AA600'


service = build('sheets', 'v4', credentials=credentials)

# Call the Sheets API
daily_log_sheet = service.spreadsheets()
daily_log_results = daily_log_sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME).execute()
daily_log_values = daily_log_results.get('values', [])

if not daily_log_values:
    print('No data found.')
    exit()

total_number_columns = len(daily_log_values[0])

validated_list_for_data_frame = []
validated_list_for_data_frame.append(daily_log_values[0])

for i in range(1, len(daily_log_values)):
    # Fill the rest of the list with empty values
    number_columns = len(daily_log_values[i])
    daily_log_values[i].extend([''] * (total_number_columns - number_columns))
    validated_list_for_data_frame.append(daily_log_values[i])

In [184]:
df = make_data_frame_from_list(validated_list_for_data_frame)


In [195]:
def calculate_fat_loss(calorie_deficit):
    CALORIES_BURN_1KG_FAT = 7700
    return (calorie_deficit/CALORIES_BURN_1KG_FAT)

def generate_summaries(df):
    today = datetime.strptime(datetime.today().strftime('%d%m%Y'), "%d%m%Y")
    monday_this_week = today - timedelta(days = today.weekday())

    until_day = datetime.today()

    monday_last_week = monday_this_week - timedelta(weeks = 1)

    # Last Week
    # 1. Average daily calorie difference from maintenance (i.e -500) 
    # 2. Average BF% decrease (-0.5%)
    # 3. Average kg decrease 

    # This month:
    # 1. Average weekly calorie diff
    # 2. Average weekly BF% decrease
    # 3. Average weekly kg decrease

    df["Body Weight Difference"] = df["Body Weight"].diff()
    df["Body Fat Difference"] = df["Body Fat"].diff()


    ##########################################################################################
    ## LAST WEEK
    ##########################################################################################
    # LAST WEEK - BODY WEEK
    
    body_weight_diff_week = df.query("Date >= @monday_last_week and Date <= @until_day") \
       .groupby(pd.Grouper(freq='W', level='Date'))['Body Weight Difference'].sum().fillna(0)

    body_weight_avg_week = df.query("Date >= @monday_last_week and Date <= @until_day") \
       .groupby(pd.Grouper(freq='W', level='Date'))['Body Weight'].mean().fillna(0)

    # LAST WEEK - BODY FAT
    body_fat_diff_week = df.query("Date >= @monday_last_week and Date <= @until_day") \
       .groupby(pd.Grouper(freq='W', level='Date'))['Body Fat Difference'].sum().fillna(0)

    body_fat_avg_week = df.query("Date >= @monday_last_week and Date <= @until_day") \
       .groupby(pd.Grouper(freq='W', level='Date'))['Body Fat'].mean().fillna(0)

    # LAST WEEK - ABOVE / BELOW MAINTENANCE
    calorie_burn_averages_weekly = df.query("Date >= @monday_last_week and Date <= @until_day") \
       .groupby(pd.Grouper(freq='W', level='Date'))['Maintenance Calorie Diff'].mean().fillna(0)

    calories_burn_maintenance_weekly = df.query("Date >= @monday_last_week and Date <= @until_day") \
        .groupby(pd.Grouper(freq='W', level='Date'))['Maintenance Calorie Diff'].sum().fillna(0)

    ##########################################################################################
    ## 4 WEEKS AGO
    ##########################################################################################
    monday_4_weeks_ago = today - timedelta(days = today.weekday(), weeks=4)

    calorie_burn_average_4_weeks = df.query("Date >= @monday_4_weeks_ago and Date <= @until_day") \
        ['Maintenance Calorie Diff'].mean()

    calories_burn_total_maintenance_4_weeks = df.query("Date >= @monday_4_weeks_ago and Date <= @until_day") \
        ['Maintenance Calorie Diff'].sum()

    body_weight_diff_4_weeks = df.query("Date >= @monday_last_week and Date <= @until_day") \
        ['Body Weight Difference'].sum()

    body_fat_diff_4_weeks = df.query("Date >= @monday_last_week and Date <= @until_day") \
        ['Body Fat Difference'].sum()


    output = {}
    output["this-week-calorie-burn-maintenance"] = calorie_burn_averages_weekly.iloc[1]
    output["this-week-calorie-burn-maintenance-sum"] = calories_burn_maintenance_weekly.iloc[1]
    output["this-week-body-kg-diff"] = body_weight_diff_week.iloc[1]
    output["this-week-body-kg-avg"] = body_weight_avg_week.iloc[1]
    output["this-week-body-fat-diff"] = body_fat_diff_week.iloc[1]
    output["this-week-body-fat-avg"] = body_fat_avg_week.iloc[1]

    output["last-week-calorie-burn-maintenance"] = calorie_burn_averages_weekly.iloc[0]
    output["last-week-calorie-burn-maintenance-sum"] = calories_burn_maintenance_weekly.iloc[0]
    output["last-week-body-kg-diff"] = body_weight_diff_week.iloc[0]
    output["last-week-body-kg-avg"] = body_weight_avg_week.iloc[0]
    output["last-week-body-fat-diff"] = body_fat_diff_week.iloc[0]
    output["last-week-body-fat-avg"] = body_fat_avg_week.iloc[0]

    output["last-4-week-calorie-burn-maintenance"] = calorie_burn_average_4_weeks
    output["last-4-week-body-kg-diff"] = calculate_fat_loss(calories_burn_total_maintenance_4_weeks)
    output["last-4-week-body-fat-diff"] = body_fat_diff_4_weeks

    json_string = json.dumps(output)

    with open('../../static/summary-data/data.json','w') as outfile:
        outfile.write(json_string)

    print(output)


generate_summaries(df)

AttributeError: 'numpy.float64' object has no attribute 'fillna'